In [1]:
%matplotlib qt5
from process_files import *
from process_darks import *
from func_pcalib import *

qt.qpa.xcb: X server does not support XInput 2


qt.qpa.xcb: QXcbConnection: XCB error: 1 (BadRequest), sequence: 169, resource id: 132, major code: 130 (Unknown), minor code: 47


In [2]:
# iline = 0
line = 'Fe_I_6173'
# calib data
config = configobj.ConfigObj('config.ini')
dkdir = config['darks']['directory']
pcdir = config['pcalibration']['directory']
settings = [f for f in os.listdir(pcdir) if 'settings' in f]
settings = pcdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
# dark frames
dk0 = data_cube(dkdir, line, 0, 0 )
dk0.data = process_dark_cube(dk0.data)
dk1 = data_cube(dkdir, line, 1, 0)
dk1.data = process_dark_cube(dk0.data)
dk2 = data_cube(dkdir, line, 2, 0)
dk2.data = process_dark_cube(dk0.data)

In [5]:
npol = int(settings['VTTControls']['ICUPolarizer\\NPositions'])
nret = int(settings['VTTControls']['ICURetarder\\NPositions'])
pang0 = float(settings['VTTControls']['ICUPolarizer\\Start'])
pang1 = float(settings['VTTControls']['ICUPolarizer\\Stop'])
poffset = float(settings['VTTControls']['ICUPolarizer\\Offset'])
pzero = float(settings['VTTControls']['ICUPolarizer\\Zero'])
pfactor = float(settings['VTTControls']['ICUPolarizer\\Factor'])
rang0 = float(settings['VTTControls']['ICURetarder\\Start'])
rang1 = float(settings['VTTControls']['ICURetarder\\Stop'])
roffset = float(settings['VTTControls']['ICURetarder\\Offset'])
rzero = float(settings['VTTControls']['ICURetarder\\Zero'])
rfactor = float(settings['VTTControls']['ICURetarder\\Factor'])
ncyc = npol*nret
ncycobs = len(os.listdir(pcdir+os.sep+line))//3
print('Intended and actual recordings of polcal data: ', ncyc, ', ', ncycobs)

# Analyze log file
logfile = [f for f in os.listdir(pcdir) if 'log' in f]
logfile = pcdir + os.sep + logfile[0]
with open(logfile) as f:
    log = f.readlines()
poltemp = [int(l.split()[-1][4:-1]) for l in log if 'X10A' in l][0:ncycobs]
rettemp = [int(l.split()[-1][4:-1]) for l in log if 'X11A' in l][0:ncycobs]
pang = np.array(poltemp)/pfactor - pzero
rang = np.array(rettemp)/rfactor - rzero
rnpos = sum(pang==pang0)
pnpos = sum(rang==rang1)
rnpos = 9
pnpos = 5
print('Retarder and polarizer positions as per logs: ', rnpos, pnpos)

Intended and actual recordings of polcal data:  45 ,  45
Retarder and polarizer positions as per logs:  9 5


In [6]:
# Get intensity
roi = 320
halfw = 640
ind = halfw-roi//2
# BBI data
int0 = []
for i in tqdm.tqdm(range(pnpos*rnpos)):
    dc = data_cube(pcdir, line, 0, i)
    int0.append(np.mean(dc.data[ind:-ind,ind:-ind,:], axis=(0,1)))
int0 = np.array(int0)
# Pol1 data
int1_mod = []
for i in tqdm.tqdm(range(pnpos*rnpos)):
    dc = data_cube(pcdir, line, 1, i)
    int1_mod.append(np.mean(dc.data[ind:-ind,ind:-ind,:], axis=(0,1)))
int1_mod = np.array(int1_mod)
# Pol2 data
int2_mod = []
for i in tqdm.tqdm(range(pnpos*rnpos)):
    dc = data_cube(pcdir, line, 2, i)
    int2_mod.append(np.mean(dc.data[ind:-ind,ind:-ind,:], axis=(0,1)))
int2_mod = np.array(int2_mod)
os.system('cls')

100%|██████████| 45/45 [02:06<00:00,  2.81s/it]
sh: 1: cls: not found


32512

In [ ]:
# Other numbers
linestr = 'Line_' + str(get_line_num(settings, line))
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
# Time stamps
tsfile = [f for f in os.listdir(pcdir) if 'timestamp' in f]
tsfile = pcdir + os.sep + tsfile[0]
ts = np.loadtxt(tsfile, delimiter=',')
im0ind, im1ind, im2ind = ts[:,3], ts[:,6], ts[:,9]
im0ind = im0ind[0:nfpc*pnpos*rnpos].reshape(int0.shape)
im1ind = im1ind[0:nfpc*pnpos*rnpos].reshape(int1_mod.shape)
im2ind = im2ind[0:nfpc*pnpos*rnpos].reshape(int2_mod.shape)
#
im0ind = im0ind - im0ind[:,0:1]
im1ind = im1ind - im1ind[:,0:1]
im2ind = im2ind - im2ind[:,0:1]
#
im0ind = im0ind.reshape([pnpos*rnpos, nwav, nacc, nmod])%4
im1ind = im1ind.reshape([pnpos*rnpos, nwav, nacc, nmod])%4
im2ind = im2ind.reshape([pnpos*rnpos, nwav, nacc, nmod])%4

In [ ]:
# Data for one cycle
b1 = np.zeros([nmod, nwav, pnpos*rnpos])
for i in range(pnpos*rnpos):
    b1[:,:,i] = coadd_modulated_intens(int1_mod[i,:], im1ind[i,:], nmod, nacc, nwav)
beam1 = np.mean(b1, axis=1)
#
b2 = np.zeros([nmod, nwav, pnpos*rnpos])
for i in range(pnpos*rnpos):
    b2[:,:,i] = coadd_modulated_intens(int2_mod[i,:], im2ind[i,:], nmod, nacc, nwav)
beam2 = np.mean(b2, axis=1)
# pnpos = 1
beam1 = beam1[:,0:pnpos*rnpos]
beam2 = beam2[:,0:pnpos*rnpos]
#
norm_int = (beam1+beam2)/np.mean(beam1+beam2)
beam1 /= norm_int
beam2 /= norm_int
#
fig, ax = plt.subplots(1,1,figsize=(10,6))
ax.plot(beam1.transpose(), ls='--')
ax.legend(['1', '2', '3', '4'])
fig.gca().set_prop_cycle(None)
ax.plot(beam2.transpose(), ls='-')
ax.plot((beam1+beam2).transpose(), color='k')
fig.tight_layout()
#

In [ ]:
# 
s_in = compute_pcu_generated_stokes(pnpos, rnpos, wpret=60)
del_dat = []
weights = beam1*0.0
weights[0,:], weights[1,:], weights[2,:], weights[3,:] = 1, 1, 1, 1
# weights = np.abs(s_in)
params = Parameters()
params.add('pa_pcusys', value=0, min=-10, max=10, vary=False)
params.add('pa_reloff', value=0, min=-10, max=10, vary=False)
params.add('wpret', value=60, min=0, max=180, vary=True)
params.add('span', value=180.0, min=-360, max=360, vary=False)
# fit_res = minimize(compute_modmat_residual, params, args=(beam1, pnpos, rnpos, weights, del_dat))
fit_res = minimize(compute_residual_twobeam, params, args=([beam1, beam2], pnpos, rnpos, weights, del_dat))
fit_res

In [ ]:
pa_pcusys, pa_reloff, wpret, span = fit_res.params.values()
thetas = np.linspace(0.0, span, pnpos)
s_fit = compute_pcu_generated_stokes(pnpos, rnpos, pa_pcusys=pa_pcusys, pa_reloff=pa_reloff, wpret=wpret)
# Beam-1
modmat_beam1 = compute_modulation_matrix(s_fit, beam1)
demodmat_beam1 = np.linalg.pinv(modmat_beam1)
meas_in1 = np.array(np.matrix(demodmat_beam1)*np.matrix(beam1))
meas_in1[1::,:] /= meas_in1[0:1,:]
meas_in1[0,:] /= np.median(meas_in1[0:1,:])
# fig, axs = plt.subplots(2,2,figsize=(10,6))
# for i, ax in enumerate(axs.flatten()):
#     ax.plot(thetas, meas_in1[i], 'k:')
#     ax.plot(thetas, s_fit[i], 'k-')
#     ax.plot(thetas, meas_in1[i]-np.mean(meas_in1[i])+np.mean(s_fit[i]), 'k--')
#     ax.set_ylim([-1.1, 1.1])
# fig.tight_layout()
print('Modulation efficiency for beam-1', compute_modulation_efficiency(demodmat_beam1))
# Beam-2
modmat_beam2 = compute_modulation_matrix(s_fit, beam2)
demodmat_beam2 = np.linalg.pinv(modmat_beam2)
meas_in2 = np.array(np.matrix(demodmat_beam2)*np.matrix(beam2))
meas_in2[1::,:] /= meas_in2[0:1,:]
meas_in2[0,:] /= np.median(meas_in2[0:1,:])
# fig, axs = plt.subplots(2,2,figsize=(10,6))
# for i, ax in enumerate(axs.flatten()):
#     ax.plot(thetas, meas_in2[i], 'k:')
#     ax.plot(thetas, s_fit[i], 'k-')
#     ax.plot(thetas, meas_in2[i]-np.mean(meas_in2[i])+np.mean(s_fit[i]), 'k--')
#     ax.set_ylim([-1.1, 1.1])
# fig.tight_layout()
print('Modulation efficiency for beam-2', compute_modulation_efficiency(demodmat_beam2))
# Plot results
xlabels = ['I', 'Q/I', 'U/I', 'V/I']
fig, axs = plt.subplots(2,2,figsize=(10,6))
for i, ax in enumerate(axs.flatten()):
    ax.plot(s_fit[i], 'k-')
    ax.plot(meas_in1[i]-np.mean(meas_in1[i])+np.mean(s_fit[i]), 'g--')
    ax.plot(meas_in2[i]-np.mean(meas_in2[i])+np.mean(s_fit[i]), 'r--')
    if (i==0): ax.set_ylim([0, 1.5])
    else: ax.set_ylim([-1.5,1.5])
    ax.set_xlabel('Wave plate angle w.r.t. polarizer (degrees)')
    ax.set_xlabel('Wave plate angle w.r.t. polarizer (degrees)')
    ax.set_ylabel(xlabels[i])
    ax.axhline(0, color='gray', ls=':')
fig.tight_layout()

In [ ]:
# int_var1 = meas_in1[i]-np.mean(meas_in1[i])+np.mean(s_fit[i])
# int_var2 = meas_in2[i]-np.mean(meas_in2[i])+np.mean(s_fit[i])
# int_var = 0.5*(int_var1[np.newaxis,:]+int_var2[np.newaxis,:])
# beam1 = beam1/int_var
# beam2 = beam2/int_var

In [ ]:
# # npos = int_mod.shape[1]
# pang_offset = params['pang_offset']
# rang_offset = params['rang_offset']
# wpret = params['wpret']
# span = params['span']
# # offset, wpret, rotang, range = params
# # s_in = compute_input_stokes(npos, offset=offset, wpret=wpret, rotang=rotang, range=range)
# s_in = compute_pcu_generated_stokes(pnpos, rnpos, pang_offset=pang_offset, rang_offset=rang_offset, wpret=wpret, span=span)

In [ ]:
#
topdir =  os.sep.join(pcdir.split(os.sep)[0:-1])
topdir += os.sep + 'L0' 
if not (os.path.exists(topdir)): os.mkdir(topdir)
dirtree = pcdir.split(os.sep)[-2::]
pc1name = topdir + os.sep + '_'.join(['HELLRIDE', 'pol1'] + dirtree + [line, 'pc.FITS'])
pc1name = pc1name.replace('PCalibration_', '')
pc2name = topdir + os.sep + '_'.join(['HELLRIDE', 'pol2'] + dirtree + [line, 'pc.FITS'])
pc2name = pc2name.replace('PCalibration_', '')
print('Modulation matrx 1 saved as: ', pc1name)
print('Modulation matrx 2 saved as: ', pc2name)
#
hdu1 = pf.PrimaryHDU(beam1)
hdu2 = pf.ImageHDU(modmat_beam1)
hdul = pf.HDUList([hdu1, hdu2])
hdul.writeto(pc1name, overwrite=True)
hdul.close()
#
hdu1 = pf.PrimaryHDU(beam2)
hdu2 = pf.ImageHDU(modmat_beam2)
hdul = pf.HDUList([hdu1, hdu2])
hdul.writeto(pc2name, overwrite=True)
hdul.close()
#
config['pcalibration'][line+'/pol1'] = pc1name
config['pcalibration'][line+'/pol2'] = pc2name
config.write()

In [ ]:
temp_ang, temp_sin = compute_input_stokes(13, wpret=61.5)
temp_modmat = ideal_mod_matrix(1)
temp_intens = get_modulation_intens(temp_sin, temp_modmat)
plt.figure()
plt.plot(temp_intens.transpose())